In [1]:
import cv2
import mediapipe as mp
import pandas as pd
from PIL import Image, ImageOps

import numpy as np
import glob, os

import matplotlib.pyplot as plt

initial_dir = os.getcwd()

df = pd.DataFrame()
frames = []

In [2]:
last_sequence_id = 0
path_past_data = f'{initial_dir}/data/data_letters.csv'
if (os.path.exists(path_past_data)):
    past_df = pd.read_csv(path_past_data)
    last_sequence_id = past_df["sequence_id"].max()

In [3]:
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode


options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='hand_landmarker.task'),
    running_mode=VisionRunningMode.IMAGE,
    num_hands=2,
)
hands = HandLandmarker.create_from_options(options)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
def normalize_coordinates(coordinates, target, image_height):
    matplot_coordinates = []
    normalized_coordinates = []
    for x, y in coordinates:
        y = image_height - y
        matplot_coordinates.append([x, y])

    # Find the minimum and maximum values among the coordinates
    min_x, min_y = np.min(matplot_coordinates, axis=0)
    max_x, max_y = np.max(matplot_coordinates, axis=0)

    # Normalize the coordinates
    normalized_coordinates = (matplot_coordinates - np.array([min_x, min_y])) / np.array([max_x - min_x, max_y - min_y])
    
    # Convert coordinates for plotting
    # visualize_data(normalized_coordinates, target)
    return normalized_coordinates

def visualize_data(normalized_coordinates, target):
    # Unzip normalized coordinates for plotting
    normalized_x, normalized_y = zip(*normalized_coordinates)
    
    # Plot the normalized coordinates
    plt.figure(figsize=(8, 6))
    plt.scatter(normalized_x, normalized_y, color='blue', label=f'{target}')
    plt.xlabel('Normalized X')
    plt.xlabel('Normalized X')
    plt.ylabel('Normalized Y')
    plt.title('Normalized Coordinates Plot')
    plt.legend()
    plt.grid(True)
    plt.show()

In [5]:
def extract_video(name, target, sequence_id):
    global frames

    # Load an image from file

    image = cv2.imdecode(np.fromfile(name, np.uint8), cv2.IMREAD_UNCHANGED)
    mp_image = mp.Image.create_from_file(name)
    hand_landmarker_result = hands.detect(mp_image)

    # Check if the image was loaded successfully
    if mp_image is not None:
        # Convert the BGR frame to RGB
        rgb_frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
        if len(hand_landmarker_result.handedness) > 0:
            if (target == "ñ"):
                target = "ñ"

            row_data = {
                "sequence_id": sequence_id,
                "target": target,
                "file": name
            }
            
            hand_sides = ["Left", "Right"]
            for idx, landmarks in enumerate(hand_landmarker_result.hand_landmarks):
                detected_pixels = []
                hand_side = hand_sides[idx]
                # Iterate through detected hand landmarks
                for landmark_idx, landmark in enumerate(landmarks):
                    x, y = landmark.x, landmark.y
                    detected_pixels.append([x, y])

                    # Draw circles on the frame
                    cv2.circle(image, (int(x * image.shape[1]), int(y * image.shape[0])), 5, (0, 255, 0), -1)

                detected_pixels = normalize_coordinates(detected_pixels, target, image.shape[0])
                # if (target == "v" or target=="y"):
        
                for i in range(len(detected_pixels)):
                    x, y = detected_pixels[i]
                    row_data[f'x_{hand_side}_hand_{i}'] =  x
                    row_data[f'y_{hand_side}_hand_{i}'] =  y
            
            if (len(hand_landmarker_result.handedness) == 1 and (target == "x" or target == "ñ" or target == "q")):
                print(name)
                return
                
            if (len(hand_landmarker_result.handedness) == 1):
                for i in range(21):
                    x, y = [0, 0]
                    row_data[f'x_{hand_sides[1]}_hand_{i}'] =  x
                    row_data[f'y_{hand_sides[1]}_hand_{i}'] =  y


            # cv2.imshow('Original', image)
            # cv2.waitKey(0)
            frames.append(row_data)
        
    else:
        print("Failed to load image.")

    cv2.destroyAllWindows()

In [6]:
sequence_id = last_sequence_id
validation_sequence_ids = []

def extract_coordinates_from_dir(dir, is_val=False):
    global sequence_id, initial_dir, validation_sequence_ids
    os.chdir(dir)

    for file in glob.glob("*"):
        if (file.split(".")[-1] != "png"):
            splitted_name = file.split(".")[:-1]
            splitted_name.append(".png")
            renamed_format = "".join(splitted_name)

            im = Image.open(file)
            im = ImageOps.exif_transpose(im)
            im.save(renamed_format)

            os.remove(file)

    files_to_extract = glob.glob("*.png")
    files_to_extract.sort()
    print(f'Hay {len(files_to_extract)} imagenes a extraer coordenadas')
    for file in files_to_extract:
        sequence_id += 1

        if (is_val):
            validation_sequence_ids.append(sequence_id)

        if ("(" not in file):
            name = file.split(".")[0]
        else:
            name = file[0:file.index("(")].strip()

        extract_video(file, name, sequence_id)

    os.chdir(initial_dir)

In [7]:
training_dir = f'{initial_dir}/data/asl_letters_lensegua_c'
extract_coordinates_from_dir(training_dir)

Hay 774 imagenes a extraer coordenadas
ñ (3) 2.png
q (1) 2.png
q (4) 2.png
q (6).png
q (7).png
x (9) 2.png
ñ (11).png
ñ (9).png


In [8]:
validation_dir = f'{initial_dir}/data/letters_val'
extract_coordinates_from_dir(validation_dir, is_val=True)

Hay 24 imagenes a extraer coordenadas


In [9]:
df = pd.DataFrame(frames)

In [10]:
df['sequence_id'] = df['sequence_id'].astype(int)

In [11]:
df.head()

,sequence_id,target,file,x_Left_hand_0,y_Left_hand_0,x_Left_hand_1,y_Left_hand_1,x_Left_hand_2,y_Left_hand_2,x_Left_hand_3,...,x_Right_hand_16,y_Right_hand_16,x_Right_hand_17,y_Right_hand_17,x_Right_hand_18,y_Right_hand_18,x_Right_hand_19,y_Right_hand_19,x_Right_hand_20,y_Right_hand_20
0,1,a,a (1) 2.png,0.000000,0.048423,0.064452,0.326796,0.334537,0.619979,0.491786,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,a,a (1) 3.png,0.000000,0.168434,0.215612,0.370625,0.470939,0.601284,0.569410,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,a,a (1).png,0.423459,0.140339,0.655401,0.363291,0.737979,0.605570,0.726230,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,a,a (10) 2.png,0.204003,0.300461,0.476328,0.473332,0.767077,0.663757,0.826010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,a,a (10) 3.png,0.920469,0.121714,0.639250,0.352007,0.432306,0.607345,0.365829,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
len(df.columns)

87

In [13]:
if (len(validation_sequence_ids) > 0):
    validation_path = f'{initial_dir}/data/validation_letters.csv'
    validation_df = df[df['sequence_id'].isin(validation_sequence_ids)][["sequence_id", "target", "file"]]
    
    train_path = f'{initial_dir}/data/train_letters.csv'
    train_df = df[~df['sequence_id'].isin(validation_sequence_ids)][["sequence_id", "target", "file"]]

    if (last_sequence_id == 0):
        validation_df.to_csv(validation_path, index=False)
        train_df.to_csv(train_path, index=False)
    else:
        df_past_val = pd.read_csv(validation_path)
        df_past_train = pd.read_csv(train_path)

        df_val_final = pd.concat([df_past_val, validation_df])
        df_train_final = pd.concat([df_past_train, train_df])

        df_val_final.to_csv(validation_path, index=False)
        df_train_final.to_csv(train_path, index=False)

In [14]:
data_path = f'{initial_dir}/data/data_letters.csv'
df_final = df.drop("file", axis=1)

if (last_sequence_id == 0):
    df_final.to_csv(data_path, index=False)
else:
    df_past_data = pd.read_csv(data_path)
    df_data_final = pd.concat([df_past_data, df_final])
    df_data_final.to_csv(data_path, index=False)

In [15]:
frames = []
extra_val_dir = f'{initial_dir}/data/test hola'
extract_coordinates_from_dir(extra_val_dir)

df = pd.DataFrame(frames) 
df['sequence_id'] = df['sequence_id'].astype(int)

Hay 4 imagenes a extraer coordenadas


In [16]:
data_path = f'{initial_dir}/data/data_test_hola.csv'
df.to_csv(data_path, index=False)